# DX 704 Week 8 Project

This homework will modify a simulator controlling a small vehicle to implement tabular q-learning.
You will first test your code with random and greedy-epsilon policies, then tweak your own training method for a more optimal policy.

The full project description and a template notebook are available on GitHub: [Project 8 Materials](https://github.com/bu-cds-dx704/dx704-project-08).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Rover Simulator

The following Python class implements a simulation of a simple vehicle with integer x,y coordinates facing in one of 8 possible directions.


In [27]:
# DO NOT CHANGE

import random

class RoverSimulator(object):
    DIRECTIONS = ((0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1), (-1, 0), (-1, 1))

    def __init__(self, resolution):
        self.resolution = resolution
        self.terminal_state = self.construct_state(resolution // 2, resolution // 2, 0)

        self.initial_states = []
        for initial_x in (0, resolution // 2, resolution - 1):
            for initial_y in (0, resolution // 2, resolution - 1):
                for initial_direction in range(8):
                    initial_state = self.construct_state(initial_x, initial_y, initial_direction)
                    if initial_state != self.terminal_state:
                        self.initial_states.append(initial_state)

    def construct_state(self, x, y, direction):
        assert 0 <= x < self.resolution
        assert 0 <= y < self.resolution
        assert 0 <= direction < 8

        state = (y * self.resolution + x) * 8 + direction
        assert self.decode_state(state) == (x, y, direction)
        return state

    def decode_state(self, state):
        direction = state % 8
        x = (state // 8) % self.resolution
        y = state // (8 * self.resolution)

        return (x, y, direction)

    def get_actions(self, state):
        return [-1, 0, 1]

    def get_next_reward_state(self, curr_state, curr_action):
        if curr_state == self.terminal_state:
            # no rewards or changes from terminal state
            return (0, curr_state)

        (curr_x, curr_y, curr_direction) = self.decode_state(curr_state)
        (curr_dx, curr_dy) = self.DIRECTIONS[curr_direction]

        assert self.construct_state(curr_x, curr_y, curr_direction) == curr_state

        assert curr_action in (-1, 0, 1)

        next_x = min(max(0, curr_x + curr_dx), self.resolution - 1)
        next_y = min(max(0, curr_y + curr_dy), self.resolution - 1)
        next_direction = (curr_direction + curr_action) % 8

        next_state = self.construct_state(next_x, next_y, next_direction)
        next_reward = 1 if next_state == self.terminal_state else 0

        return (next_reward, next_state)

    def rollout_policy(self, policy_func, max_steps=1000):
        curr_state = self.sample_initial_state()
        for i in range(max_steps):
            curr_action = policy_func(curr_state, self.get_actions(curr_state))
            (next_reward, next_state) = self.get_next_reward_state(curr_state, curr_action)
            yield (curr_state, curr_action, next_reward, next_state)
            curr_state = next_state

    def sample_initial_state(self):
        return random.choice(self.initial_states)

In [28]:
simulator = RoverSimulator(16)
initial_sample = simulator.sample_initial_state()
print("INITIAL SAMPLE", initial_sample)

INITIAL SAMPLE 5


## Part 1: Implement a Random Policy

Random policies are often used to test simulators and start initial exploration.
Implement a random policy for these simulators.

In [29]:
# YOUR CHANGES HERE

def random_policy(state, actions):
    return random.choice(actions)

Use the code below to test your random policy.
Then modify it to save the results in "log-random.tsv" with the columns curr_state, curr_action, next_reward and next_state.

In [30]:
# YOUR CHANGES HERE

for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(random_policy, max_steps=32):
    print("CURR STATE", curr_state, "ACTION", curr_action, "NEXT REWARD", next_reward, "NEXT STATE", next_state)

CURR STATE 1926 ACTION -1 NEXT REWARD 0 NEXT STATE 1925
CURR STATE 1925 ACTION 0 NEXT REWARD 0 NEXT STATE 1797
CURR STATE 1797 ACTION 0 NEXT REWARD 0 NEXT STATE 1669
CURR STATE 1669 ACTION 0 NEXT REWARD 0 NEXT STATE 1541
CURR STATE 1541 ACTION -1 NEXT REWARD 0 NEXT STATE 1412
CURR STATE 1412 ACTION 0 NEXT REWARD 0 NEXT STATE 1284
CURR STATE 1284 ACTION 0 NEXT REWARD 0 NEXT STATE 1156
CURR STATE 1156 ACTION -1 NEXT REWARD 0 NEXT STATE 1027
CURR STATE 1027 ACTION 1 NEXT REWARD 0 NEXT STATE 908
CURR STATE 908 ACTION -1 NEXT REWARD 0 NEXT STATE 779
CURR STATE 779 ACTION 1 NEXT REWARD 0 NEXT STATE 660
CURR STATE 660 ACTION 0 NEXT REWARD 0 NEXT STATE 532
CURR STATE 532 ACTION 1 NEXT REWARD 0 NEXT STATE 405
CURR STATE 405 ACTION 0 NEXT REWARD 0 NEXT STATE 269
CURR STATE 269 ACTION 1 NEXT REWARD 0 NEXT STATE 134
CURR STATE 134 ACTION 0 NEXT REWARD 0 NEXT STATE 134
CURR STATE 134 ACTION 0 NEXT REWARD 0 NEXT STATE 134
CURR STATE 134 ACTION 0 NEXT REWARD 0 NEXT STATE 134
CURR STATE 134 ACTION 1 N

In [31]:
import csv
with open("log-random.tsv", "w", newline="") as f:
    writer = csv.writer(f, delimiter="\t")
    writer.writerow(["curr_state", "curr_action", "next_reward", "next_state"])
    
    for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(random_policy, max_steps=32):
        writer.writerow([curr_state, curr_action, next_reward, next_state])
        print("CURR STATE", curr_state, "ACTION", curr_action,
              "NEXT REWARD", next_reward, "NEXT STATE", next_state)

CURR STATE 0 ACTION 0 NEXT REWARD 0 NEXT STATE 128
CURR STATE 128 ACTION -1 NEXT REWARD 0 NEXT STATE 263
CURR STATE 263 ACTION 1 NEXT REWARD 0 NEXT STATE 384
CURR STATE 384 ACTION 0 NEXT REWARD 0 NEXT STATE 512
CURR STATE 512 ACTION 0 NEXT REWARD 0 NEXT STATE 640
CURR STATE 640 ACTION -1 NEXT REWARD 0 NEXT STATE 775
CURR STATE 775 ACTION -1 NEXT REWARD 0 NEXT STATE 902
CURR STATE 902 ACTION 0 NEXT REWARD 0 NEXT STATE 902
CURR STATE 902 ACTION 1 NEXT REWARD 0 NEXT STATE 903
CURR STATE 903 ACTION -1 NEXT REWARD 0 NEXT STATE 1030
CURR STATE 1030 ACTION 1 NEXT REWARD 0 NEXT STATE 1031
CURR STATE 1031 ACTION -1 NEXT REWARD 0 NEXT STATE 1158
CURR STATE 1158 ACTION 0 NEXT REWARD 0 NEXT STATE 1158
CURR STATE 1158 ACTION 0 NEXT REWARD 0 NEXT STATE 1158
CURR STATE 1158 ACTION 1 NEXT REWARD 0 NEXT STATE 1159
CURR STATE 1159 ACTION 1 NEXT REWARD 0 NEXT STATE 1280
CURR STATE 1280 ACTION -1 NEXT REWARD 0 NEXT STATE 1415
CURR STATE 1415 ACTION -1 NEXT REWARD 0 NEXT STATE 1542
CURR STATE 1542 ACTION 0

Submit "log-random.tsv" in Gradescope.

## Part 2: Implement Q-Learning with Random Policy

The code below runs 32 random rollouts of 1024 steps using your random policy.
Modify the rollout code to implement Q-Learning.
Just implement one learning update for each sampled state-action in the simulation.
Use $\alpha=1$ and $\gamma=0.9$ since the simulator is deterministic and there is a sink where the rewards stop.




In [32]:
# YOUR CHANGES HERE

for episode in range(32):
    for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(random_policy, max_steps=1024):
        #print("CURR STATE", curr_state, "ACTION", curr_action, "NEXT REWARD", next_reward, "NEXT STATE", next_state)
        pass

Save each step in the simulator in a file "q-random.tsv" with columns curr_state, curr_action, next_reward, next_state, old_value, new_value.

In [33]:
# YOUR CHANGES HERE

from collections import defaultdict

Q = defaultdict(lambda: {a: 0.0 for a in simulator.get_actions(0)})

alpha = 1.0      
gamma = 0.9     

with open("q-random.tsv", "w", newline="") as f:
    writer = csv.writer(f, delimiter="\t")
    writer.writerow(["curr_state", "curr_action", "next_reward", "next_state", "old_value", "new_value"])

    for episode in range(32):
        for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(random_policy, max_steps=1024):

            old_value = Q[curr_state][curr_action]

            next_max_q = max(Q[next_state].values())

            new_value = next_reward + gamma * next_max_q

            Q[curr_state][curr_action] = new_value

            writer.writerow([curr_state, curr_action, next_reward, next_state, old_value, new_value])

Submit "q-random.tsv" in Gradescope.

## Part 3: Implement Epsilon-Greedy Policy

Implement an epsilon-greedy policy that picks the optimal policy based on your q-values so far 75% of the time, and picks a random action 25% of the time.
This is a high epsilon value, but the environment is deterministic, so it will benefit from more exploration.

In [43]:
# YOUR CHANGES HERE

# hard-code epsilon=0.25. this is high but the environment is deterministic.

epsilon = 0.25
def epsilon_greedy_policy(state, actions):
    if random.random() < epsilon:
        return random.choice(actions)
    best_action = max(actions, key=lambda a: Q[state][a])
    return best_action

Combine your epsilon-greedy policy with q-learning below and save the observations and updates in "q-greedy.tsv" with columns curr_state, curr_action, next_reward, next_state, old_value, new_value.

Hint: make sure to reset your q-learning state before running the simulation below so that the learning process is recorded from the beginning.

In [44]:
# YOUR CHANGES HERE

Q = defaultdict(lambda: {a: 0.0 for a in simulator.get_actions(0)})

alpha = 1.0
gamma = 0.9

with open("q-greedy.tsv", "w", newline="") as f:
    writer = csv.writer(f, delimiter="\t")
    writer.writerow(["curr_state", "curr_action", "next_reward", "next_state", "old_value", "new_value"])

    for episode in range(32):
        for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(epsilon_greedy_policy, max_steps=1024):

            old_value = Q[curr_state][curr_action]
            next_max_q = max(Q[next_state].values())

            new_value = next_reward + gamma * next_max_q
            Q[curr_state][curr_action] = new_value

            writer.writerow([curr_state, curr_action, next_reward, next_state, old_value, new_value])

            if next_reward > 0:
                break

Submit "q-greedy.tsv" in Gradescope.

## Part 4: Extract Policy from Q-Values

Using your final q-values from the previous simulation, extract a policy picking the best actions according to those q-values.
Save the policy in a file "policy-greedy.tsv" with columns state and action.

In [45]:
# YOUR CHANGES HERE

with open("policy-greedy.tsv", "w", newline="") as f:
    writer = csv.writer(f, delimiter="\t")
    writer.writerow(["state", "action"])

    for state in Q.keys():
        best_action = max(Q[state], key=lambda a: Q[state][a])
        writer.writerow([state, best_action])

Submit "policy-greedy.tsv" in Gradescope.

## Part 5: Implement Large Policy

Train a more optimal policy using q-learning.
Save the policy in a file "policy-optimal.tsv" with columns state and action.

Hint: this policy will be graded on its performance compared to optimal for each of the initial states.
**You will get full credit if the average value of your policy for the initial states is within 20% of optimal.**
Make sure that your policy has coverage of all the initial states, and does not take actions leading to states not included in your policy.
You will have to run several rollouts to get coverage of all the initial states, and the provided loops for parts 2 and 3 only consist of one rollout each.

Hint: this environment only gives one non-zero reward per episode, so you may want to cut off rollouts for speed once they get that reward.
But make sure you update the q-values first!

In [47]:
# YOUR CHANGES HERE

Q = defaultdict(lambda: {a: 0.0 for a in simulator.get_actions(0)})

alpha = 1.0
gamma = 0.9
episodes = 1000  

epsilon_start = 0.5
epsilon_end = 0.05

def epsilon_greedy_policy(state, actions, epsilon):
    if random.random() < epsilon:
        return random.choice(actions)
    return max(actions, key=lambda a: Q[state][a])

for episode in range(episodes):
    epsilon = max(epsilon_end, epsilon_start * (0.99 ** episode))  
    for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(
        lambda s, a: epsilon_greedy_policy(s, a, epsilon), max_steps=1024
    ):
        old_value = Q[curr_state][curr_action]
        next_max = max(Q[next_state].values())
        new_value = next_reward + gamma * next_max
        Q[curr_state][curr_action] = new_value

        if next_reward > 0: 
            break
        
with open("policy-optimal.tsv", "w", newline="") as f:
    writer = csv.writer(f, delimiter="\t")
    writer.writerow(["state", "action"])
    for state in Q.keys():
        best_action = max(Q[state], key=lambda a: Q[state][a])
        writer.writerow([state, best_action])

Submit "policy-optimal.tsv" in Gradescope.

## Part 6: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 7: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.

None